In [7]:
from keras.layers import Input, Flatten, Dense
from keras.layers.merge import concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils.vis_utils import plot_model

In [73]:
import feather
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [72]:
StandardScaler?

In [26]:
# import data

In [86]:
flights = feather.read_dataframe('../data/flights-2008-sample.feather')

In [87]:
# define X, y

In [88]:
X = flights[['DayOfWeek', 'DayofMonth', 'Month', 'ScheduledDepTimestamp', 'Origin', 'Dest', 'UniqueCarrier']].copy()
y = flights['total_delay'].copy()

In [89]:
# encode categorical variables as value counts

In [90]:
value_counts = {col: X[col].value_counts() for col in X.columns if col != 'ScheduledDepTimestamp'}
for col, val_counts in value_counts.items():
    X.loc[:, col] = X[col].map(val_counts)

In [91]:
# normalize X columns, log-transform y

In [92]:
X.isnull().sum()

DayOfWeek                0
DayofMonth               0
Month                    0
ScheduledDepTimestamp    0
Origin                   0
Dest                     0
UniqueCarrier            0
dtype: int64

In [95]:
X = StandardScaler().fit_transform(X)
y = np.log(y + 1).values